In [1]:
from bs4 import BeautifulSoup
#from  selenium import webdriver
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import psycopg2
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings("ignore")

In [2]:
url = "https://www.cars.com/shopping/results/?makes[]=mercedes_benz&maximum_distance=30&models[]=&page=2&stock_type=all&zip=%27"

In [3]:
def get_soup(url):
    page_source = requests.get(url)
    soup = BeautifulSoup(page_source.content,"html.parser")
    return soup

In [4]:
soup = get_soup(url)

In [5]:
all_elt = soup.find_all("div",class_="vehicle-card-main js-gallery-click-card")

In [6]:
all_elt[0].find("span",class_="sds-rating__count").text

'4.6'

In [7]:
def get_data(url):
    soup = get_soup(url)
    all_elt = soup.find_all("div",class_="vehicle-card-main js-gallery-click-card")
    titles = []
    prices = []
    #img_links = []
    dealer_names = []
    dealer_phones = []
    infos = []
    trims = []
    makes = []
    customer_ids  = []
    stock_types = []
    listing_ids = []
    model_years = []
    models = []
    sponsoreds = []
    msrps = []
    sponsored_types = []
    pricess = []
    bodystyles = []
    results_page_numbers = []
    cpo_indicators = []
    vertical_positions = [] 
    notes = []
    f = lambda x : x.text.strip("\n") if x else "-"
    g = lambda x : x if x else "-"
    h = lambda x : x["href"].strip("tel:") if x else '-'
    for elt in all_elt:
        title = elt.find("h2",class_="title").text
        price = elt.find("span",class_="primary-price").text
        #img_link = elt.find('img',class_="vehicle-image js-lazy-load styles-lazy-load")["data-src"]
        info = elt.find("a",class_="sds-link")["data-override-payload"]
        dealer_name = elt.find("div",class_="dealer-name")
        dealer_phone = elt.find("div", class_="contact-buttons").find("a", class_="sds-button--secondary contact-by-phone")
        note = elt.find("span",class_="sds-rating__count")
        notes.append(f(note))
        titles.append(title)
        prices.append(price)
        #img_links.append(g(img_link))
        infos.append(info)
        dealer_names.append(f(dealer_name))
        dealer_phones.append(h(dealer_phone))
        info = json.loads(info)
        trims.append(info['trim'])
        makes.append(info['make'])
        customer_ids.append(info['customer_id'])
        stock_types.append(info['stock_type'])
        listing_ids.append(info['listing_id'])
        model_years.append(info['model_year'])
        models.append(info['model'])
        sponsoreds.append(info['sponsored'])
        msrps.append(info[f'msrp'])
        sponsored_types.append(info['sponsored_type'])
        pricess.append(info['price'])
        bodystyles.append(info['bodystyle'])
        results_page_numbers.append(info['results_page_number'])
        cpo_indicators.append(info['cpo_indicator'])
        vertical_positions.append(info['vertical_position'])
        
    data = pd.DataFrame({"Nom":titles,"Prix":prices,"Dealer Phone":dealer_phones,"Dealer Name":dealer_names,"trims":trims,"makes":makes,"customer_ids":customer_ids,"stock_types":stock_types,"model_years" : model_years,"models" : models,"sponsoreds" : sponsoreds,"msrps" : msrps,"sponsored_types": sponsored_types,"prices" : pricess,"bodystyles" : bodystyles,"results_page_numbers" : results_page_numbers,"cpo_indicators" : cpo_indicators,"vertical_positions" : vertical_positions,"Rating":notes})
    return data
#     print(len(titles))
#     print(len(prices))
#     print(len(img_links))
#     print(len(dealer_names))
#     print(len(dealer_phones))
#     print(len(infos ))
#     print(len(trims))
#     print(len(makes))
#     print(len(customer_ids))
#     print(len(stock_types))
#     print(len(listing_ids))
#     print(len(model_years))
#     print(len(models))
#     print(len(sponsoreds))
#     print(len(msrps))
#     print(len(sponsored_types))
#     print(len(prices))
#     print(len(bodystyles))
#     print(len(results_page_numbers))
#     print(len(cpo_indicators))
#     print(len(vertical_positions))

# "Lien":img_links,
                              
#         print(f'Nom : {title}')
#         print(f'Prix : {price}')
#         print(f'Lien :  {img_link}')
#         print(f'Info : {json.loads(info)}')
#         print(f'Dealer name : {dealer_name}')
#         print(f'Dealer Phone : {dealer_phone}')
#         print(f'Lien : {str(img_link)[str(img_link).index("data-src")+10:]}')
#         print("======================================================================")

In [8]:
data = get_data(url)

In [9]:
data

,Nom,Prix,Dealer Phone,Dealer Name,trims,makes,customer_ids,stock_types,model_years,models,sponsoreds,msrps,sponsored_types,prices,bodystyles,results_page_numbers,cpo_indicators,vertical_positions,Rating
0,2024 Mercedes-Benz Maybach S 580 4MATIC,"$190,450",+1-732-372-4427,Mercedes-Benz of Edison,4matic,mercedes_benz,188776,new,2024,maybach_s_580,False,199450,standard,190450,sedan,1,False,2,5.0
1,2024 Mercedes-Benz C-Class C 300 4MATIC,"$50,295",+1-281-206-0598,Mercedes-Benz of West Houston,c_300_4matic,mercedes_benz,5372042,new,2024,c_class,False,50295,standard,50295,sedan,1,False,3,4.6
2,2024 Mercedes-Benz Sprinter 2500 High Roof,"$73,851",+1-843-407-2653,Mercedes-Benz Van Center - Baker,high_roof,mercedes_benz,5396442,new,2024,sprinter_2500,False,73851,standard,73851,cargo_van,1,False,4,4.6
3,2023 Mercedes-Benz AMG GLA 35 Base,"$43,400",+1-888-771-1141,Mercedes-Benz of Bellevue,base,mercedes_benz,158589,used,2023,amg_gla_35,False,None,standard,43400,suv,1,False,5,4.6
4,2024 Mercedes-Benz C-Class C 300,"$54,650",+1-888-901-0695,Mercedes-Benz of Buckhead,c_300,mercedes_benz,149170,new,2024,c_class,False,54650,standard,54650,sedan,1,False,6,3.6
5,2024 Mercedes-Benz GLB 250 Base,"$49,100",+1-888-901-0695,Mercedes-Benz of Buckhead,base,mercedes_benz,149170,new,2024,glb_250,False,49100,standard,49100,suv,1,False,7,3.6
6,2024 Mercedes-Benz Sprinter 2500 High Roof,"$114,106",+1-843-407-2653,Mercedes-Benz Van Center - Baker,high_roof,mercedes_benz,5396442,new,2024,sprinter_2500,False,114106,standard,114106,cargo_van,1,False,8,4.6
7,2024 Mercedes-Benz Sprinter 2500 High Roof,"$64,132",+1-843-407-2653,Mercedes-Benz Van Center - Baker,high_roof,mercedes_benz,5396442,new,2024,sprinter_2500,False,64132,standard,64132,cargo_van,1,False,9,4.6
8,2019 Mercedes-Benz AMG C 43 Base 4MATIC,"$37,977",+1-877-254-2461,European Auto Expo,base_4matic,mercedes_benz,190513,used,2019,amg_c_43,False,None,standard,37977,sedan,1,False,10,3.0
9,2018 Mercedes-Benz S-Class S 560,"$44,995",+1-866-953-5842,Mercedes-Benz of Coconut Creek,s_560,mercedes_benz,204368,used,2018,s_class,False,None,standard,44995,sedan,1,True,11,4.5


In [10]:
def get_data_bypage(size,nb_pages):
    """Cette fonction prend en paramètres un nombre données de pages à investiguer 
    et renvoie un dataframe contenant les différentes informations"""
    # Initialisation d'un nouveau dataframe
    final_data = pd.DataFrame(columns= ["Nom","Prix","Dealer Phone","Dealer Name","trims","makes","customer_ids","stock_types","model_years" ,"models" ,"sponsoreds" ,"msrps" ,"sponsored_types","prices" ,"bodystyles" ,"results_page_numbers" ,"cpo_indicators" ,"vertical_positions" ,"Lien"])
    for p in range(1,nb_pages+1) :
        # Url type selon le numéro de la page
        url_type = f"https://www.cars.com/shopping/results/?makes[]=mercedes_benz&maximum_distance=30&models[]=&page={p}&page_size={size}&stock_type=all&zip=%27"
        # Rechercher les informations sur la page
        data_find = get_data(url_type)
        # Ajouter les informations trouvées au dataframe initié au début
        final_data = pd.concat([final_data,data_find], axis=0)
        # Réinitialiser les index du dataframe
        final_data = final_data.reset_index(drop=True)
    return final_data

In [11]:
data = get_data_bypage(100,20)

In [12]:
data

,Nom,Prix,Dealer Phone,Dealer Name,trims,makes,customer_ids,stock_types,model_years,models,sponsoreds,msrps,sponsored_types,prices,bodystyles,results_page_numbers,cpo_indicators,vertical_positions,Lien,Rating
0,2023 Mercedes-Benz CLS 450 Base 4MATIC,"$83,140",+1-877-221-5668,Mercedes-Benz of Laguna Niguel,base_4matic,mercedes_benz,85645,new,2023,cls_450,False,83140,standard,83140.0,sedan,1,False,2,NaN,4.7
1,2016 Mercedes-Benz S-Class S 550 4MATIC,"$44,500",+1-540-709-5070,Alpha Motorsports Fredericksburg,s_550_4matic,mercedes_benz,209201,used,2016,s_class,False,None,standard,44500.0,coupe,1,False,3,NaN,4.7
2,2015 Mercedes-Benz M-Class ML 350 4MATIC,"$14,798",+1-630-608-2663,BG Motorcars,ml_350_4matic,mercedes_benz,191154,used,2015,m_class,False,None,standard,14798.0,suv,1,False,4,NaN,4.0
3,2024 Mercedes-Benz Maybach GLS 600 4MATIC,"$175,500",+1-281-206-0598,Mercedes-Benz of West Houston,4matic,mercedes_benz,5372042,new,2024,maybach_gls_600,False,175500,standard,175500.0,suv,1,False,5,NaN,4.6
4,2023 Mercedes-Benz AMG EQS Base,"$162,030",+1-847-562-5154,Autohaus on Edens,base,mercedes_benz,38,new,2023,amg_eqs,False,162030,standard,162030.0,sedan,1,False,6,NaN,4.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,2023 Mercedes-Benz EQE 350 Base 4MATIC,"$98,480",+1-888-380-2546,Aristocrat Motors,base_4matic,mercedes_benz,81163,new,2023,eqe_350,False,98480,standard,98480.0,suv,1,False,97,NaN,4.7
1994,2024 Mercedes-Benz GLS 580 4MATIC,"$118,560",+1-336-773-9984,Mercedes-Benz of Charlottesville,4matic,mercedes_benz,6062388,new,2024,gls_580,False,118560,standard,118560.0,suv,1,False,98,NaN,4.8
1995,2023 Mercedes-Benz EQS 580 Base 4MATIC,"$131,125",+1-888-434-9416,Mercedes-Benz of Plano,base_4matic,mercedes_benz,21711,new,2023,eqs_580,False,131125,standard,131125.0,suv,1,False,99,NaN,4.7
1996,2024 Mercedes-Benz S-Class Base,"$143,180",+1-609-524-0005,Mercedes-Benz of Princeton,base,mercedes_benz,98984,new,2024,s_class,False,143180,standard,143180.0,sedan,1,False,100,NaN,4.9


In [13]:
data.columns

Index(['Nom', 'Prix', 'Dealer Phone', 'Dealer Name', 'trims', 'makes',
       'customer_ids', 'stock_types', 'model_years', 'models', 'sponsoreds',
       'msrps', 'sponsored_types', 'prices', 'bodystyles',
       'results_page_numbers', 'cpo_indicators', 'vertical_positions', 'Lien',
       'Rating'],
      dtype='object')

In [14]:
def connect_to_postgresql(user, password, host, port, database):
    
    try:
        # Création d'une connexion à la base de données PostgreSQL
        conn = psycopg2.connect(user=user, password=password, host=host, port=port, database=database)

        # Création d'un moteur SQLAlchemy
        engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

        return conn, engine

    except Exception as e:
        print(f"Erreur lors de la connexion à la base de données PostgreSQL : {e}")
        raise  # Vous pouvez choisir de gérer l'exception d'une manière spécifique à votre application

In [15]:
def insert_data_to_postgres(data, table_name='cars'):
    try:
    
        # Insérer les données dans la base de données PostgreSQL en évitant les doublons
        data.to_sql(table_name, engine, if_exists='append', index=False, method='multi', chunksize=1000)

        # Commit de la transaction
        conn.commit()

        print("Data retrieved and inserted into PostgreSQL")

    except Exception as e:
        if 'unique constraint' in str(e):
            print("Les doublons ont été détectés et ne seront pas insérés.")
        else:
            print(f"Une erreur s'est produite lors de l'insertion dans PostgreSQL : {e}")


In [16]:
# Fonction principale
def get_data_bypage_real_time():
    size = 100
    nb_pages = 20

    while True:
        try:
            # Faire une requête pour récupérer les données du site
            response = requests.get("https://www.cars.com/shopping/results/?makes[]=mercedes_benz&maximum_distance=30&models[]=&page=2&stock_type=all")
            
            if response.status_code == 200:
                # Traiter les données ici
                data = get_data_bypage(size, nb_pages)

                # Insérer les données dans la base de données PostgreSQL
                insert_data_to_postgres(data)
                print("Data retrieved and inserted into PostgreSQL")

            else:
                print(f"Échec de la requête. Code d'état : {response.status_code}")

        except Exception as ex:
            print(f"An error occurred: {ex}")

        # Attendre 5 secondes avant de faire la prochaine requête
        time.sleep(1000)


In [17]:
# Exemple d'utilisation de la fonction
try:
    db_config = {
        'user': 'postgres',
        'password': 'lola1205',
        'host': 'localhost',
        'port': '5432',
        'database': 'postgres',
    }

    conn, engine = connect_to_postgresql(**db_config)
    
    get_data_bypage_real_time()

    
finally:
    # Fermer la connexion lorsqu'elle n'est plus nécessaire
    if conn:
        conn.close()


Data retrieved and inserted into PostgreSQL
Data retrieved and inserted into PostgreSQL
Data retrieved and inserted into PostgreSQL
Data retrieved and inserted into PostgreSQL
Data retrieved and inserted into PostgreSQL
Data retrieved and inserted into PostgreSQL
Data retrieved and inserted into PostgreSQL
Data retrieved and inserted into PostgreSQL
Data retrieved and inserted into PostgreSQL
Data retrieved and inserted into PostgreSQL
Data retrieved and inserted into PostgreSQL
Data retrieved and inserted into PostgreSQL


KeyboardInterrupt: 